In [14]:
import pandas as pd

data = {'ID': [1, 2, 3, 4, 5, 6], 'Name': ['John', 'Jane', 'Mary', 'John1', 'Jane1', 'Mary1'], 'AniType': ['Lion', 'Monkey', 'Lion', 'Lion', 'Monkey', 'Horse'], 'Gender': ['M', 'F', 'F', 'M', 'M', 'F'], 'Weight': [10, 100, 50, 10, 100, 50]}
df = pd.DataFrame(data)
df

,ID,Name,AniType,Gender,Weight
0,1,John,Lion,M,10
1,2,Jane,Monkey,F,100
2,3,Mary,Lion,F,50
3,4,John1,Lion,M,10
4,5,Jane1,Monkey,M,100
5,6,Mary1,Horse,F,50


In [15]:
df.groupby('AniType')['AniType'].count()

AniType
Horse     1
Lion      3
Monkey    2
Name: AniType, dtype: int64

In [13]:
df[df['Gender'] == 'M'].groupby('AniType')['AniType'].count()

AniType
Lion    3
Name: AniType, dtype: int64

In [17]:
df2 = df.groupby('AniType')['Gender'].value_counts()
df2

AniType  Gender
Horse    F         1
Lion     M         2
         F         1
Monkey   F         1
         M         1
Name: Gender, dtype: int64

In [20]:
# df2.append([['Horse', 'M'], 0])
df2 = pd.DataFrame(df2)
df2.loc[('Horse', 'M')] = ('Horse', 'M', 0)
df2

AttributeError: 'DataFrame' object has no attribute 'ix'

In [24]:
df[df['Gender'] == 'M'].groupby('AniType')['AniType'].count()

AniType
Lion      2
Monkey    1
Name: AniType, dtype: int64

In [25]:
df[df['Gender'] == 'F'].groupby('AniType')['AniType'].count()

AniType
Horse     1
Lion      1
Monkey    1
Name: AniType, dtype: int64

In [31]:
m = pd.DataFrame(df[df['Gender'] == 'M'].groupby('AniType')['AniType'].count())
f = pd.DataFrame(df[df['Gender'] == 'F'].groupby('AniType')['AniType'].count())
m

,AniType
AniType,
Lion,2
Monkey,1


In [33]:
m = m.rename(columns={"AniType": "M"})


In [37]:
m

,M
AniType,
Lion,2
Monkey,1


In [34]:
f = f.rename(columns={"AniType": "F"})
f

In [38]:
f

,F
AniType,
Horse,1
Lion,1
Monkey,1


In [35]:
dfm = pd.merge(m, f)
dfm

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [36]:
m.merge(f)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [40]:
df3 = pd.concat([m, f], axis=1, join='outer')
df3.fillna(0, inplace=True)
df3

,M,F
AniType,,
Lion,2.0,1
Monkey,1.0,1
Horse,0.0,1


In [1]:
import json

diff = {'add': ['5-6', '9-10'], 'remove': ['5-4', '9-4']}
with open("diff.json", "w") as file:
    json.dump(diff, file)

In [1]:
from esa import SAW

sa = SAW(r"C:\Users\test\PWCase\IEEE 14_bus_20220628.PWB")

In [3]:
key = sa.get_key_field_list('branch')
fields = key + ['Selected']
df = sa.GetParametersMultipleElement('branch', fields)
df

,BusNum,BusNum:1,LineCircuit,Selected
0,1,2,1,NO
1,1,5,1,NO
2,2,3,1,NO
3,2,4,1,NO
4,2,5,1,NO
5,3,4,1,NO
6,4,5,1,NO
7,4,7,1,NO
8,4,9,1,NO
9,5,6,1,NO


In [4]:
df['Selected'] = 'YES'
sa.change_parameters_multiple_element_df('branch', df)
sa.GetParametersMultipleElement('branch', fields)

,BusNum,BusNum:1,LineCircuit,Selected
0,1,2,1,YES
1,1,5,1,YES
2,2,3,1,YES
3,2,4,1,YES
4,2,5,1,YES
5,3,4,1,YES
6,4,5,1,YES
7,4,7,1,YES
8,4,9,1,YES
9,5,6,1,YES


In [5]:
sa.RunScriptCommand('CalculateShiftFactorsMultipleElement(BRANCH,SELECTED,BUYER,[SLACK],DC)')

In [15]:
sa.pw_order = True
sa.GetParametersMultipleElement('Bus', ['MultBusTLRSens', 'MultBusTLRSens:1']).to_numpy(dtype=float)

array([[ 0.        ,  0.        ],
       [-0.83802629, -0.1619737 ],
       [-0.74654102, -0.25345895],
       [-0.6675052 , -0.33249477],
       [-0.61055624, -0.38944376],
       [-0.62992203, -0.37007794],
       [-0.65729469, -0.34270534],
       [-0.65729469, -0.34270534],
       [-0.6519233 , -0.3480767 ],
       [-0.64801323, -0.35198674],
       [-0.63912565, -0.36087433],
       [-0.63166058, -0.36833942],
       [-0.63301903, -0.36698097],
       [-0.64365786, -0.35634214]])

In [16]:
df.shape

(20, 4)

In [17]:
isf_fields = ['MultBusTLRSens']
for i in range(1, df.shape[0]):
    isf_fields += [f"MultBusTLRSens:{i}"]

In [18]:
sa.GetParametersMultipleElement('Bus', isf_fields).to_numpy(dtype=float)

array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-8.3802629e-01, -1.6197370e-01,  2.7343450e-02,  5.7223980e-02,
         7.7406270e-02,  2.7343450e-02,  8.0020670e-02,  2.8890500e-03,
         1.6577100e-03, -4.5467600e-03, -2.7379500e-03, -4.0213000e-04,
        -1.4066700e-03,  0.0000000e+00,  2.8890500e-03,  2.7379500e-03,
         1.8088100e-03,  2.7379500e-03, -4.0213000e-04, -1.8088100e-03],
       [-7.4654102e-01, -2.5345895e-01, -5.3203267e-01, -1.4343266e-01,
        -7.1075740e-02,  4.6796736e-01,  3.0708614e-01,  1.1086980e-02,
         6.3615900e-03, -1.7448570e-02, -1.0507120e-02, -1.5432200e-03,
        -5.3982300e-03,  0.0000000e+00,  1.1086980e-02,  1.050